In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
import os
import json
import time
# Census API Key
from config import api_key
from config import g_key
import gmaps
import requests
import gmplot
import time
from scipy.stats import linregress
from pprint import pprint
import seaborn as sns

ModuleNotFoundError: No module named 'gmplot'

In [ ]:
#census_data = c.acs5.get().json()
#Purpose is to test Query url and review the results of the URL in a borwser to better understand the data
#And validate the URL
query_url = f"https://api.census.gov/data/2019/acs/acs5"
display(query_url)
#https://api.census.gov/data#/2019/acs/acs5&{api_key}#?get=NAME,group(B01001)&for=us:1&key={api_key}

In [ ]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

#Loop created to create and join the 2014 ACS 5 year an 2019 ACS 5 year data.

cols = ["Zipcode", "Household Income", "Population",
        "Median Contract Rent", "Median Gross Rent", "Median Home Value",
        "Median Monthly Owner Costs", "Year"]


years = [2014, 2019]
#years = [2015, 2015, 2016, 2017, 2018, 2019]
#years = [2019]
for year in years:

    c = Census(api_key, year=year)
    census_data = c.acs5.get(("NAME", "B19013_001E", 
                              "B01003_001E",
                              "B25058_001E",
                              "B25064_001E", 
                              "B25077_001E", 
                              "B25088_002E"), {'for': 'zip code tabulation area:*'})

    # Convert to DataFrame
    census_pd = pd.DataFrame(census_data)

    # Column Reordering
    census_pd = census_pd.rename(columns={"B19013_001E": f"Household Income_{year}",
                                          "B01003_001E": f"Population_{year}",
                                          "B25058_001E": f"Median Contract Rent_{year}",
                                          "B25064_001E": f"Median Gross Rent_{year}",
                                          "B25077_001E": f"Median Home Value_{year}",
                                          "B25088_002E": f"Median Monthly Owner Costs_{year}",
                                          "NAME": "Name",
                                          "zip code tabulation area": "Zipcode"})
    
    census_pd = census_pd.drop(columns=['Name'])
    # Final DataFrame
    
    
    if year==2014:
        df = census_pd
    if year==2019:
        df = df.merge(census_pd, how='inner', on=['Zipcode', 'state'])

display(df.head())

In [ ]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
df.to_csv("census_data.csv", encoding="utf-8", index=False)

In [ ]:
!ls

In [ ]:
#create DataFrame for MSA
#https://www.roelpeters.be/solved-dtypewarning-columns-have-mixed-types-specify-dtype-option-on-import-or-set-low-memory-in-pandas/
msa_df = pd.read_csv('../project_1/files/ScanUSZipCode2017A.csv',low_memory=False)
msa_df

In [ ]:
#Comparing Data Types between the two diffrent dataframes to make certain they match for the merge
print(msa_df.dtypes)

In [ ]:
print(df.dtypes) 
    

In [ ]:
#convert Zipcode datatype in concat_df to be int64
#https://www.kite.com/python/answers/how-to-convert-a-pandas-dataframe-column-from-object-to-int-in-python

df["Zipcode"] = df["Zipcode"].astype(object).astype(int)

In [ ]:
#Rename ZIP column in MSA to match Zipcode from Census data
#https://note.nkmk.me/en/python-pandas-dataframe-rename/

msa_df.rename(columns={'ZIP': 'Zipcode'}, inplace=True)


In [ ]:
#Merge data frames and drop the values in the census data with -666666666
#Some cells at the intesection of rows and colums had -666666666 , which represents missing, null ot incomplete data.
#Those rows (zipcodes) with any data such as this were dropped from the dataset

merged_census_df = pd.merge(df, msa_df, how="left", on=["Zipcode", "Zipcode"])
merged_census_df
merged_census_df.drop(merged_census_df[merged_census_df["Household Income_2014"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Population_2014"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Median Contract Rent_2014"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Median Gross Rent_2014"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Median Home Value_2014"] == -666666666].index, inplace = True)       
merged_census_df.drop(merged_census_df[merged_census_df["Median Monthly Owner Costs_2014"] == -666666666].index, inplace = True) 
merged_census_df.drop(merged_census_df[merged_census_df["Household Income_2019"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Population_2019"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Median Contract Rent_2019"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Median Gross Rent_2019"] == -666666666].index, inplace = True)
merged_census_df.drop(merged_census_df[merged_census_df["Median Home Value_2019"] == -666666666].index, inplace = True)       
merged_census_df.drop(merged_census_df[merged_census_df["Median Monthly Owner Costs_2019"] == -666666666].index, inplace = True)

 

In [ ]:
# Save as a csv to check full data set
# Note to avoid any issues later, use encoding="utf-8"
merged_census_df.to_csv("census_data_2014_2019Years.csv", encoding="utf-8", index=False)

In [ ]:
# Remove zips with no MA
#Data cleansing step in which zipcodes that did not map to a MSA were dropped
merged_census_df['MA'].replace('', np.nan, inplace = True)
merged_census_df.dropna(subset=['MA'], inplace=True)
merged_census_df.head()

In [ ]:
# Create dataframe of home ownership
#Data exported and reviewed as part of the projects ongoing data validation and quality review

msa_home_ownership = pd.read_csv('../project_1/files/Census_Home_Ownership.csv',low_memory=False)
msa_home_ownership


In [ ]:
# Merge the ACS 5 Year census home ownership with existing data
full_census_merge = pd.merge(merged_census_df, msa_home_ownership, how="left", on=["MANAME", "MANAME"])
full_census_merge

In [ ]:
#Filter out zips not contained in top 75 largest MSAs
full_census_merge.dropna(inplace=True)
full_census_merge

In [ ]:
# Check curreent dataframe types
#Required for current and future merges.  Merges can only occur on the same dtypes.
print(full_census_merge.dtypes)

In [ ]:
#Initial merging of ACS 5 yeat data set for 2014 and 2019.
# Convert median monthly owner cost to integer
#The main data frame is called df.  Also, the columns are the original names witn _2014 and _2019 as a suffix
full_census_merge["Median Monthly Owner Costs_2014"] = full_census_merge["Median Monthly Owner Costs_2014"].astype(object).astype(float)
full_census_merge["Median Monthly Owner Costs_2019"] = full_census_merge["Median Monthly Owner Costs_2019"].astype(object).astype(float)

In [ ]:
# Confirm data type conversion
#Data type conversion had to occur earlier to merge data.
#This is a subsequent quality and process check to vaidate results
print(full_census_merge.dtypes)

In [ ]:
# Create homeowner % averages
full_census_merge["Average H%_2014"] = full_census_merge[['2010 H%','2011 H%','2012 H%','2013 H%','2014 H%']].mean(axis=1)
full_census_merge["Average H%_2019"] = full_census_merge[['2015 H%','2016 H%','2017 H%','2018 H%','2019 H%']].mean(axis=1)

In [ ]:
# Create weighted cost column
full_census_merge["Weighted Cost_2014"] = full_census_merge["Median Gross Rent_2014"] * (1-full_census_merge["Average H%_2014"] / 100) + full_census_merge["Median Monthly Owner Costs_2014"] * full_census_merge["Average H%_2014"] / 100
full_census_merge["Weighted Cost_2019"] = full_census_merge["Median Gross Rent_2019"] * (1-full_census_merge["Average H%_2019"] / 100) + full_census_merge["Median Monthly Owner Costs_2019"] * full_census_merge["Average H%_2019"] / 100
full_census_merge

# Create weighted cost column
full_census_merge["Weighted Cost_2014"] = full_census_merge["Median Gross Rent_2014"] * (1-full_census_merge["Average H%_2014"] / 100) + full_census_merge["Median Monthly Owner Costs_2014"] * full_census_merge["Average H%_2014"] / 100
full_census_merge["Weighted Cost_2019"] = full_census_merge["Median Gross Rent_2019"] * (1-full_census_merge["Average H%_2019"] / 100) + full_census_merge["Median Monthly Owner Costs_2019"] * full_census_merge["Average H%_2019"] / 100
full_census_merge

In [ ]:
# Create Affordability Index
full_census_merge["Affordability_2014"] = full_census_merge["Weighted Cost_2014"] / full_census_merge["Household Income_2014"] * 12
full_census_merge["Affordability_2019"] = full_census_merge["Weighted Cost_2019"] / full_census_merge["Household Income_2019"] * 12
full_census_merge

In [ ]:
#Ecport of data for validation and data quality checks
full_census_merge.to_csv("Full_Census_Merge.csv", encoding="utf-8", index=False)

In [ ]:
# Add population growth metrics by zip code
full_census_merge["Population Growth"] = full_census_merge["Population_2019"] - full_census_merge["Population_2014"]
full_census_merge["Population Growth %"] = full_census_merge["Population Growth"] / full_census_merge["Population_2014"]

In [ ]:
# Add change in income metric by zip code
full_census_merge["Change in Income"] = full_census_merge["Household Income_2019"] - full_census_merge["Household Income_2014"]
full_census_merge["Change in Income %"] = full_census_merge["Change in Income"] / full_census_merge["Household Income_2014"]

In [ ]:
# Add change in Weighted Cost
full_census_merge["Change in Weighted Cost"] = full_census_merge["Weighted Cost_2019"] - full_census_merge["Weighted Cost_2014"]
full_census_merge["Change in Weighted Cost %"] = full_census_merge["Change in Weighted Cost"] / full_census_merge["Weighted Cost_2014"]
full_census_merge

In [ ]:
# Add change in Affordability
full_census_merge["Change in Affordability"] = full_census_merge["Affordability_2019"] - full_census_merge["Affordability_2014"]
full_census_merge["Change in Affordability %"] = full_census_merge["Change in Affordability"] / full_census_merge["Affordability_2014"]

In [ ]:
#Export of full census merge data after the affordability and change in affordability were calculated
#Export was to allow validation and data quality work to be conducted

full_census_merge.to_csv("Full_Census_Merge_Metrics.csv", encoding="utf-8", index=False)

In [ ]:
#Used groupby function on the cleaned full census data clean to create calculations used in the later data frames, calculations
#and Analysi

MSA_agg = full_census_merge.groupby(['MANAME']).agg(
    Zip_count = ('Zipcode','count'),
    Population_2014 = ('Population_2014','sum'),
    Population_2019 = ('Population_2019','sum'),
    Population_growth = ('Population Growth','sum'),
    Affordability_2019 = ('Affordability_2019','mean'),
    Income_Growth = ('Change in Income %', 'mean'),
    Weighted_Cost_Change = ('Change in Weighted Cost %','mean'),
    Affordability_Change = ('Change in Affordability %','mean')
).reset_index()    

MSA_agg

In [ ]:
#Calcuation of the Population Growth Percentage Metric
MSA_agg["Population_growth_%"] = MSA_agg["Population_growth"] / MSA_agg["Population_2014"]
MSA_agg

In [ ]:
MSA_agg.to_csv("MSA_Agg.csv", encoding="utf-8", index=False)

In [ ]:
#Data Frame of the MSA withs top growth by %
#Data frame used later with Atlanta data for charting and regression analysis

MSA_top_growth=MSA_agg.nlargest(5,'Population_growth_%')
MSA_top_growth

In [ ]:
#Data Frame to include atlanta in top 5 comparison
#Data Frame referenced/used in later charting
MSA_top_growth_atl = MSA_top_growth.append(MSA_agg[MSA_agg["MANAME"]=="Atlanta-Sandy Springs-Roswell GA"])
MSA_top_growth_atl

In [ ]:
#Create DF for Bar Graph to identify 5 slowest growing MSAs by population % change.
#Note: all 5 of the bottom have negative growth and affordability

MSA_bottom_growth = MSA_agg.nsmallest(5,'Population_growth_%')
MSA_bottom_growth

In [ ]:
#Create DF for Bar Graph to identify 5 least affordable

MSA_least_affordable=MSA_agg.nlargest(5,'Affordability_2019')
MSA_least_affordable

In [ ]:
# Export CSV's for analysis
#THis was a data validation and analysis step.  
#This validation allowed the team to identify the Top 5 growing MSA by growth, bottom 5 MSAs by growth and Least affordable MSAs

MSA_top_growth_atl.to_csv("MSA_top_growth_atl.csv", encoding="utf-8", index=False)
MSA_bottom_growth.to_csv("MSA_bottom_growth.csv", encoding="utf-8", index=False)
MSA_least_affordable.to_csv("MSA_least_affordable.csv", encoding="utf-8", index=False)

In [ ]:
#Top 5 MSA identied through the above analysis that met the study criteria
top_msa_name = ["Austin-Round Rock TX", "Orlando-Kissimmee-Sanford FL", "Raleigh NC", "Houston-The Woodlands-Sugar Land TX", "San Antonio-New Braunfels TX", "Atlanta-Sandy Springs-Roswell GA"]


In [ ]:
#Merging the initial data after cleaning from the census API for the ACS 5 Year with the MSA data with the list of MSAs we identified
#as being in the top 5 as it related to % prowth and Affordability.
#The MSA data assigned many to most of the zip codes to a MSA.
#Merge using the isin function.
#Data frame printed to validate merge results

msa_top_growth_zip = full_census_merge[full_census_merge.MANAME.isin(top_msa_name)]
msa_top_growth_zip

In [ ]:
#Creation and export of the MSAs with the top growh out to CSV.
#As we learned and worked with our datasets, we chose to periodicially export the data and leverage other tools
#To validat our results and initial conclusions generated inside the noteboook
#NOTE: Date validation and review were a significant commitment and required significiant project time.

msa_top_growth_zip.to_csv("msa_top_growth_zip.csv", encoding="utf-8", index=False)

In [ ]:
#Data Frame created to include atlanta in top 5 comparison of MSAs having largest Growth
#Atlanta is this teams home and base for this bootcamp.  Thus, we wanted to also compare Atlanta
MSA_top_growth_atl = MSA_top_growth.append(MSA_agg[MSA_agg["MANAME"]=="Atlanta-Sandy Springs-Roswell GA"])
MSA_top_growth_atl

In [ ]:
#Needed to loop and anotate the points on the graph with the associated MSAs
text = list(MSA_top_growth_atl['MANAME'])
print(text)

In [ ]:
#Linear Regression and Scatter plot to measure the relationship bewtween change in population and Change in affordability
#view of data is by MSA
#Chart uses the msa_top_growth_zip DataFrame created earlier 

x_values = pd.to_numeric(MSA_top_growth_atl["Population_growth_%"]) #.astype(float)
y_values = pd.to_numeric(MSA_top_growth_atl["Affordability_Change"]) #.astype(float)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " +str(round(intercept,2))
# plot for northern hemisphere - max temp v/ latitude linear regression
fig1,ax1 = plt.subplots(figsize=(15,10))
plt.scatter(x_values, y_values, alpha = 0.90, edgecolors = 'k', linewidths = 1)
plt.plot(x_values, regress_values, "r--")
plt.annotate(line_eq,(.10 ,-.04),fontsize=14)
plt.xlabel("Population Growth %",fontsize=20)
plt.ylabel("Affordability Change",fontsize=20)
plt.title(f"Change in population % vs Change in Affordability by MSA",fontsize=25)
sns.set(style="whitegrid")
#calculations of p/r values
print(f"The P-Value is: {pvalue}")
print(f"The R-Value is: {rvalue}")
print(f"The r-squared is: {rvalue **2}")

text = list(MSA_top_growth_atl['MANAME'])
xvals = list(x_values)
yvals = list(y_values)
for i in range(len(xvals)):
    plt.annotate(text[i], (xvals[i], yvals[i]+0.003))


#Save the image
plt.savefig("files/ChangePopvChangeAffordbyMSA.png",bbox_inches = "tight")
 
plt.show()

## Change in population % vs Change in Affordability by MSA with Regression

In [ ]:
#Linear Regression and Scatter plot to measure the relationship bewtween change in population and Change in affordability
#view of data is by zipcode
#Chart uses the msa_top_growth_zip DataFrame created earlier

x_values = pd.to_numeric(msa_top_growth_zip["Population Growth %"]) 
y_values = pd.to_numeric(msa_top_growth_zip["Change in Affordability"])

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " +str(round(intercept,2))

fig1,ax1 = plt.subplots(figsize=(20,15))
plt.scatter(x_values, y_values, alpha = 0.90, s = 22, edgecolors = 'k', linewidths = 5)
plt.plot(x_values, regress_values, "r--")

#plt.annotate(line_eq,(10 ,15),fontsize=14)

plt.xlabel("Zipcode",fontsize=20)
plt.ylabel("Affordability Change",fontsize=20)
plt.title(f"Change in Population Growth vs Change in Affordability by Zipcode",fontsize=30)
sns.set(style="whitegrid")

#calculations of p/r values
print(f"The P-Value is: {pvalue}")
print(f"The R-Value is: {rvalue}")
print(f"The r-squared is: {rvalue **2}")

plt.show()

#Save the image
plt.savefig("files/ChangeInPopGrowthvAffordChangebyZip.png", bbox_inches = "tight")
 
plt.show()

In [ ]:
#Zips only to be used in the Geocoding via the Google API 

msa_top_growth_zip_only=msa_top_growth_zip['Zipcode'].reset_index(drop=True)
msa_top_growth_zip_only
msa_top_growth_zip_only


In [ ]:
 query_url = f'https://maps.googleapis.com/maps/api/geocode/json?address=30309=&sensor=true&key={g_key}'
 display(query_url)   

#I have written this a 1,000 ways to Sunday and still cannot get this to generate the google map with zipcode points.
#The URL returned is the correct URL for the zipcode
#I am missing genertaing the map and cannot figure out had to add postal code as a lyer

#Google Maps of our Cities
import gmaps
#figure = gmaps
zips = msa_top_growth_zip_only
i=len(msa_top_growth_zip_only)
count=0
#if count = i:
if count < 4:
    for zip in zips:

        query_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={zip}=&sensor=true&key={g_key}'
        display(query_url)
        result = requests.get(query_url).json()  
        figure = gmaps.figure(zoom_level=3,center=(48.8566,2.3522))
        
        
count =+1
display(count)

In [ ]:
msa_top_growth_zip_only=msa_top_growth_zip['Zipcode'].reset_index(drop=True)
msa_top_growth_zip_only

In [ ]:
text_bottom = list(MSA_bottom_growth['MANAME'])
print(text_bottom)

In [ ]:
#Linear 
x_values = pd.to_numeric(MSA_bottom_growth["Population_growth_%"]) #.astype(float)
y_values = pd.to_numeric(MSA_bottom_growth["Affordability_Change"]) #.astype(float)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " +str(round(intercept,2))
# plot for northern hemisphere - max temp v/ latitude linear regression
fig1,ax1 = plt.subplots(figsize=(15,10))
plt.scatter(x_values, y_values, alpha = 0.90, edgecolors = 'k', linewidths = 1)
plt.plot(x_values, regress_values, "r--")
#plt.annotate(line_eq,(.10 ,-.04),fontsize=14)
plt.xlabel("Population Growth %",fontsize=20)
plt.ylabel("Affordability Change",fontsize=20)
plt.title(f"Negative Population % Growth vs Change in Affordability",fontsize=25)
sns.set(style="whitegrid")
#calculations of p/r values
print(f"The P-Value is: {pvalue}")
print(f"The R-Value is: {rvalue}")
print(f"The r-squared is: {rvalue **2}")

text_bottom = list(MSA_bottom_growth['MANAME'])
xvals = list(x_values)
yvals = list(y_values)
for i in range(len(xvals)):
    plt.annotate(text_bottom[i], (xvals[i], yvals[i]+0.002))
#save plot as png


#Save the image
plt.savefig("files/NegativePopGrowthLinearRegression.png")
plt.show()